<a href="https://colab.research.google.com/github/dharanipriyaR24/SDGnE/blob/main/SDGnE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install sdgne

In [ ]:
!pip install gretel-client

In [ ]:
pip install pandas scikit-learn imbalanced-learn

In [ ]:
!pip install --upgrade --force-reinstall numpy
!pip install --upgrade --force-reinstall pandas

  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 2.2.4 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.2.4 which is incompatible.
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 

In [ ]:
!pip install pandas

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import pairwise_distances
from imblearn.over_sampling import SMOTE

In [ ]:
df = pd.read_csv('/content/SB-078_original.csv')

In [ ]:
df.head()

,id,date,age,sex,BMI,income,road_dist,cooking,am_pef_org,y_am_pef,...,windsd,humid,varp,dewpt,airp,seap,solrhr,solramnt,grdt,class
0,SB-078,2017-12-19,78.0,1.0,23.873,0.143,0.5,1.0,350.0,1.000,...,0.499,0.578,0.179,0.467,0.812,0.810,0.450,0.135,0.202,1.0
1,SB-078,2017-12-20,78.0,1.0,23.873,0.143,0.5,1.0,350.0,0.875,...,0.349,0.320,0.069,0.268,1.000,1.000,0.682,0.223,0.134,1.0
2,SB-078,2017-12-21,78.0,1.0,23.873,0.143,0.5,1.0,350.0,0.875,...,0.276,0.725,0.172,0.462,0.922,0.919,0.264,0.128,0.202,1.0
3,SB-078,2017-12-22,78.0,1.0,23.873,0.143,0.5,1.0,350.0,0.875,...,0.166,0.532,0.169,0.465,0.907,0.903,0.704,0.231,0.236,1.0
4,SB-078,2017-12-23,78.0,1.0,23.873,0.143,0.5,1.0,350.0,0.875,...,0.377,0.697,0.291,0.607,0.729,0.723,0.433,0.146,0.288,1.0


In [ ]:
print("Imbalance Count:")
print(df['class'].value_counts())

Imbalance Count:
class
1.0    107
0.0     46
Name: count, dtype: int64


In [ ]:
# STEP 1: Identify majority and minority class
class_counts = df['class'].value_counts()
majority_class = class_counts.idxmax()
minority_class = class_counts.idxmin()

majority_idx = df[df['class'] == majority_class].index
minority_idx = df[df['class'] == minority_class].index

In [ ]:
# STEP 2: Hide 25% of the majority class
hidden_size = int(len(majority_idx) * 0.25)
hidden_majority = majority_idx[:hidden_size]
visible_majority = majority_idx[hidden_size:]


In [ ]:
# Combine visible majority and minority for oversampling
visible_indices = visible_majority.union(minority_idx)
X_visible = df.loc[visible_indices].drop(columns=['class', 'id'])  # Drop ID and label
y_visible = df.loc[visible_indices]['class']

# Convert categorical columns to numeric (if any)
X_visible = pd.get_dummies(X_visible, drop_first=True)

In [ ]:
# Step 2: Apply Standard SMOTE to Balance the Data
# -------------------------------
smote = SMOTE(random_state=42)
# Use X_visible and y_visible instead of X and y
X_resampled, y_resampled = smote.fit_resample(X_visible, y_visible)

# Calculate the number of synthetic samples generated
num_original = len(X_visible) # Use X_visible here
num_resampled = len(X_resampled)
num_synthetic = num_resampled - num_original
print("Number of synthetic samples generated:", num_synthetic)


Number of synthetic samples generated: 35


In [ ]:
# STEP 4: Extract synthetic samples only
new_samples_count = len(X_resampled) - len(X_visible)
X_synthetic = X_resampled[-new_samples_count:]
y_synthetic = y_resampled[-new_samples_count:]

In [ ]:
# STEP 5: Bring back hidden majority and original minority
X_hidden_majority = df.loc[hidden_majority].drop(columns=['class', 'id'])
X_minority = df.loc[minority_idx].drop(columns=['class', 'id'])

In [ ]:
# Encode hidden majority and minority to match synthetic structure
X_hidden_majority = pd.get_dummies(X_hidden_majority, drop_first=True)
X_minority = pd.get_dummies(X_minority, drop_first=True)

# Align column structures
X_hidden_majority = X_hidden_majority.reindex(columns=X_synthetic.columns, fill_value=0)
X_minority = X_minority.reindex(columns=X_synthetic.columns, fill_value=0)


In [ ]:
ed_to_minority = pairwise_distances(X_synthetic, X_minority).min(axis=1)
ed_to_majority = pairwise_distances(X_synthetic, X_hidden_majority).min(axis=1)
ed_error = (ed_to_majority < ed_to_minority).sum() / len(X_synthetic)
print(f"Euclidean Validation Error: {ed_error:.4f}")

Euclidean Validation Error: 0.0000


In [ ]:
# Hassanat distance between two values
def D(p_i, q_i):
    if min(p_i, q_i) >= 0:
        return 1 - (1 + min(p_i, q_i)) / (1 + max(p_i, q_i))
    else:
        return 1 - (1 + min(p_i, q_i) + abs(min(p_i, q_i))) / (1 + max(p_i, q_i) + abs(min(p_i, q_i)))

# Hassanat distance between two vectors
def hassanat_distance(p, q):
    return sum(D(p_i, q_i) for p_i, q_i in zip(p, q))

# Vectorized version: distance from each row in A to each row in B
def hassanat_pairwise(A, B):
    dist_matrix = np.zeros((A.shape[0], B.shape[0]))
    for i in range(A.shape[0]):
        for j in range(B.shape[0]):
            dist_matrix[i, j] = hassanat_distance(A.iloc[i], B.iloc[j])
    return dist_matrix


In [ ]:
# Use Hassanat distance

hd_to_minority = hassanat_pairwise(X_synthetic, X_minority)
hd_to_majority = hassanat_pairwise(X_synthetic, X_hidden_majority)

dist_to_minority = hd_to_minority.min(axis=1)
dist_to_majority = hd_to_majority.min(axis=1)


In [ ]:
# STEP 7: Validation Error
closer_to_majority = (dist_to_majority < dist_to_minority).sum()
total_synthetic = len(X_synthetic)
error_rate = closer_to_majority / total_synthetic

In [ ]:
# Result
print(f"Validation Error: {error_rate:.4f} ({closer_to_majority}/{total_synthetic} synthetic samples were closer to the majority class)")


Validation Error: 0.0000 (0/35 synthetic samples were closer to the majority class)
